In [2]:
!pip install -q git+https://github.com/openai/whisper.git > /dev/null --use-deprecated=legacy-resolver
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null --use-deprecated=legacy-resolver

In [3]:
!apt-get update
!apt-get install -y nvidia-driver-470

!nvidia-smi

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [668 kB]
Get:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,678 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.co

In [4]:
import os
import whisper
import datetime
import subprocess
import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))
from pyannote.audio import Audio
from pyannote.core import Segment
import wave
import contextlib
from sklearn.cluster import AgglomerativeClustering
import numpy as np
from functools import reduce

/usr/local/lib/python3.10/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
/usr/local/lib/python3.10/dist-packages/torch_audiomentations/utils/io.py:27: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


hyperparams.yaml:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

In [38]:
os.makedirs('audioFiles/'+'12345', exist_ok=True)
os.makedirs('audioFiles/'+'12346', exist_ok=True)
os.makedirs('audioFiles/'+'12347', exist_ok=True)

In [39]:
def getAllFiles():
    audioFilesDict = {}
    directory = 'audioFiles'
    for root, _, files in os.walk(directory):
          for file in files:
            file_path = os.path.join(root, file)
            file_name = os.path.basename(file_path)
            audioFilesDict[file_path] = file

    return audioFilesDict

In [45]:
filesDict = getAllFiles()

In [46]:
filesDict

{'audioFiles/12346/harvard.wav': 'harvard.wav',
 'audioFiles/12347/audio.wav': 'audio.wav',
 'audioFiles/12345/jackhammer.wav': 'jackhammer.wav'}

In [47]:
num_speakers = 2 #@param {type:"integer"}
language = 'English' #@param ['any', 'English']
model_size = 'large' #@param ['tiny', 'base', 'small', 'medium', 'large']
model_name = model_size
if language == 'English' and model_size != 'large':
  model_name += '.en'

In [48]:
for filePath in filesDict.keys():
  if filesDict[filePath][-3:] != 'wav':
    subprocess.call(['ffmpeg', '-i', filesDict[filePath], filesDict[filePath]+'.wav', '-y'])
    filesDict[filePath] = filesDict[filePath]+'.wav'


In [11]:
model = whisper.load_model(model_size)

100%|█████████████████████████████████████| 2.88G/2.88G [00:50<00:00, 60.8MiB/s]


In [49]:
segments = []
audioFrameRateDict = {}
for filePath in filesDict.keys():
  print(filesDict[filePath])
  result = model.transcribe(filePath)
  result["segments"] = list(map(lambda d:{**d, filePath: filesDict[filePath]},result["segments"]))
  segments.append(result["segments"])
  with contextlib.closing(wave.open(filePath,'r')) as f:
    audioFrameRateDict[filePath] = {'frames':f.getnframes(), 'rate':f.getframerate(), 'duration':f.getnframes() / float(f.getframerate())}

harvard.wav
audio.wav
jackhammer.wav


In [50]:
segments

[[{'id': 0,
   'seek': 0,
   'start': 0.0,
   'end': 4.0,
   'text': ' The stale smell of old beer lingers.',
   'tokens': [50365,
    440,
    342,
    1220,
    4316,
    295,
    1331,
    8795,
    22949,
    433,
    13,
    50565],
   'temperature': 0.0,
   'avg_logprob': -0.1422352434983894,
   'compression_ratio': 1.4210526315789473,
   'no_speech_prob': 0.10358467698097229,
   'audioFiles/12346/harvard.wav': 'harvard.wav'},
  {'id': 1,
   'seek': 0,
   'start': 4.0,
   'end': 7.0,
   'text': ' It takes heat to bring out the odor.',
   'tokens': [50565, 467, 2516, 3738, 281, 1565, 484, 264, 41176, 13, 50715],
   'temperature': 0.0,
   'avg_logprob': -0.1422352434983894,
   'compression_ratio': 1.4210526315789473,
   'no_speech_prob': 0.10358467698097229,
   'audioFiles/12346/harvard.wav': 'harvard.wav'},
  {'id': 2,
   'seek': 0,
   'start': 7.0,
   'end': 10.0,
   'text': ' A cold dip restores health and zest.',
   'tokens': [50715,
    316,
    3554,
    10460,
    1472,
    

In [51]:
audioFrameRateDict

{'audioFiles/12346/harvard.wav': {'frames': 809508,
  'rate': 44100,
  'duration': 18.356190476190477},
 'audioFiles/12347/audio.wav': {'frames': 4214701,
  'rate': 48000,
  'duration': 87.80627083333333},
 'audioFiles/12345/jackhammer.wav': {'frames': 147590,
  'rate': 44100,
  'duration': 3.3467120181405896}}

In [53]:
audio = Audio(sample_rate=16000, mono="downmix")
def segment_embedding(segment,duration,path):
  start = segment["start"]
  # Whisper overshoots the end timestamp in the last segment
  end = min(duration, segment["end"])
  clip = Segment(start, end)
  waveform, sample_rate = audio.crop(path, clip)
  return embedding_model(waveform[None])

In [54]:
#segmentList = reduce(lambda x,y : x+y,segments)
embeddingsDict = {}
for segmentList in segments:
  segmentPath = ""
  embeddings = np.zeros(shape=(len(segmentList), 192))
  for i, segment in enumerate(segmentList):
    path,fileName = list(segment.items())[-1]
    duration = audioFrameRateDict[path]["duration"]
    segmentPath = path
    print(path)
    print(embeddings)
    embeddings[i] = segment_embedding(segment,duration,path)
  embeddings = np.nan_to_num(embeddings)
  embeddingsDict[segmentPath] = embeddings

audioFiles/12346/harvard.wav
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
audioFiles/12346/harvard.wav
[[ 11.59518528  -1.3906852    3.38086557 ...  18.5696106  -11.58891392
    4.21837378]
 [  0.           0.           0.         ...   0.           0.
    0.        ]
 [  0.           0.           0.         ...   0.           0.
    0.        ]
 [  0.           0.           0.         ...   0.           0.
    0.        ]
 [  0.           0.           0.         ...   0.           0.
    0.        ]
 [  0.           0.           0.         ...   0.           0.
    0.        ]]
audioFiles/12346/harvard.wav
[[ 11.59518528  -1.3906852    3.38086557 ...  18.5696106  -11.58891392
    4.21837378]
 [-47.00848389  21.39778709  -0.05302324 ...  22.8205719  -42.85869217
   17.22029305]
 [  0.           0.           0.         ...   0.           0.
    0.        ]
 [  0.           0.       

In [55]:
pathSegmentListDict={}
for segmentList in segments:
    for i, segment in enumerate(segmentList):
      pathSegmentListDict[list(segment)[-1]] = segmentList

print(pathSegmentListDict)

{'audioFiles/12346/harvard.wav': [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 4.0, 'text': ' The stale smell of old beer lingers.', 'tokens': [50365, 440, 342, 1220, 4316, 295, 1331, 8795, 22949, 433, 13, 50565], 'temperature': 0.0, 'avg_logprob': -0.1422352434983894, 'compression_ratio': 1.4210526315789473, 'no_speech_prob': 0.10358467698097229, 'audioFiles/12346/harvard.wav': 'harvard.wav'}, {'id': 1, 'seek': 0, 'start': 4.0, 'end': 7.0, 'text': ' It takes heat to bring out the odor.', 'tokens': [50565, 467, 2516, 3738, 281, 1565, 484, 264, 41176, 13, 50715], 'temperature': 0.0, 'avg_logprob': -0.1422352434983894, 'compression_ratio': 1.4210526315789473, 'no_speech_prob': 0.10358467698097229, 'audioFiles/12346/harvard.wav': 'harvard.wav'}, {'id': 2, 'seek': 0, 'start': 7.0, 'end': 10.0, 'text': ' A cold dip restores health and zest.', 'tokens': [50715, 316, 3554, 10460, 1472, 2706, 1585, 293, 37889, 13, 50865], 'temperature': 0.0, 'avg_logprob': -0.1422352434983894, 'compression_ratio'

In [57]:
for path in embeddingsDict.keys():
  print("*"*20)
  print(len(embeddingsDict[path]))
  if len(embeddingsDict[path]) == 1:
    for i in range(len(pathSegmentListDict[path])):
      print(path)
      last_underscore_index = pathSegmentListDict[path][i][path].rfind("_")
      dot_index = pathSegmentListDict[path][i][path].rfind(".")
      pathSegmentListDict[path][i]["speaker"] = 'SPEAKER 1'
      pathSegmentListDict[path][i]["dateTime"] = pathSegmentListDict[path][i][path][last_underscore_index + 1:dot_index]
  else:
    clustering = AgglomerativeClustering(num_speakers).fit(embeddingsDict[path])
    labels = clustering.labels_
    print(labels)
    for i in range(len(pathSegmentListDict[path])):
      last_underscore_index = pathSegmentListDict[path][i][path].rfind("_")
      dot_index = pathSegmentListDict[path][i][path].rfind(".")
      pathSegmentListDict[path][i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)
      pathSegmentListDict[path][i]["dateTime"] = pathSegmentListDict[path][i][path][last_underscore_index + 1:dot_index]
print(pathSegmentListDict)

********************
6
[0 1 0 1 0 1]
********************
35
[0 1 0 1 0 1 1 1 0 0 0 1 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
********************
1
audioFiles/12345/jackhammer.wav
{'audioFiles/12346/harvard.wav': [{'id': 0, 'seek': 0, 'start': 0.0, 'end': 4.0, 'text': ' The stale smell of old beer lingers.', 'tokens': [50365, 440, 342, 1220, 4316, 295, 1331, 8795, 22949, 433, 13, 50565], 'temperature': 0.0, 'avg_logprob': -0.1422352434983894, 'compression_ratio': 1.4210526315789473, 'no_speech_prob': 0.10358467698097229, 'audioFiles/12346/harvard.wav': 'harvard.wav', 'speaker': 'SPEAKER 1', 'dateTime': 'harvard'}, {'id': 1, 'seek': 0, 'start': 4.0, 'end': 7.0, 'text': ' It takes heat to bring out the odor.', 'tokens': [50565, 467, 2516, 3738, 281, 1565, 484, 264, 41176, 13, 50715], 'temperature': 0.0, 'avg_logprob': -0.1422352434983894, 'compression_ratio': 1.4210526315789473, 'no_speech_prob': 0.10358467698097229, 'audioFiles/12346/harvard.wav': 'harvard.wav', 'speaker': 'SPEA

In [58]:
import os
import json

for key in pathSegmentListDict.keys():
  firstIndex = key.rfind("/")
  lastIndex = key.rfind(".")
  os.makedirs('diarizedFiles/'+key.split("/")[2], exist_ok=True)
  file_path = 'diarizedFiles/'+key.split("/")[2]
  dataFile = json.dumps(pathSegmentListDict[key], indent=2)
  with open(file_path+"/"+key[firstIndex + 1 : lastIndex]+".json", "w+") as f:
    json.dump(dataFile, f)
